# Lab 05 Logistic Classification (diabetes)
* Logistic Classfication을 diabetes data를 활용하여 모델을 만들어 보도록 하겠습니다
### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input

tf.random.set_seed(0)   # for reproducibility
print(tf.__version__)

2.12.0


### 강의에 설명할 Data입니다

In [2]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)

(759, 8) (759, 1)
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


##  Tensorflow
### 위 Data를 기준으로 가설의 검증을 통해 Logistic Classification 모델을 만들도록 하겠습니다
* Tensorflow data API를 통해 학습시킬 값들을 담는다 (Batch Size는 한번에 학습시킬 Size로 정한다)

In [3]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

### 위 Data를 기준으로 가설의 검증을 통해 Logistic Classification 모델을 만들도록 하겠습니다
* W와 b은 학습을 통해 생성되는 모델에 쓰이는 Wegith와 Bias (초기값을 variable : 0이나 Random값으로 가능 tf.random_normal([2, 1]) )

In [5]:
W = tf.Variable(tf.random.normal([8, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

### Sigmoid 함수를 가설로 선언합니다
* Sigmoid는 아래 그래프와 같이 0과 1의 값만을 리턴합니다 tf.sigmoid(tf.matmul(X, W) + b)와 같습니다

$$
\begin{align}
sigmoid(x) & = \frac{1}{1+e^{-x}}  \\\\\
\end{align}
$$

![sigmoid](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)

In [6]:
def logistic_regression(features):
    hypothesis  = tf.math.sigmoid(tf.matmul(features, W) + b)
    return hypothesis

### 가설을 검증할 Cost 함수를 정의합니다
$$
\begin{align}
cost(h(x),y) & = −log(h(x))  &  if  &  y=1 \\\\\
cost(h(x),y) & = -log(1−h(x))  &  if  &  y=0
\end{align}
$$

* 위 두수식을 합치면 아래과 같습니다
$$
\begin{align}
cost(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [19]:
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)

### 추론한 값은 0.5를 기준(Sigmoid 그래프 참조)로 0과 1의 값을 리턴합니다.
* Sigmoid 함수를 통해 예측값이 0.5보다 크면 1을 반환하고 0.5보다 작으면 0으로 반환합니다.

In [38]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

### GradientTape를 통해 경사값을 계산합니다.

In [17]:
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features,labels)
    return tape.gradient(loss_value, [W,b])

### Tensorflow를 통해 실행합니다.
* 위의 Data를 Cost함수를 통해 학습시킨 후 모델을 생성합니다. 

In [20]:
EPOCHS = 2001

for step in range(EPOCHS):
    for features, labels in iter(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),features,labels)))

Iter: 0, Loss: 0.6533
Iter: 100, Loss: 0.6448
Iter: 200, Loss: 0.6367
Iter: 300, Loss: 0.6291
Iter: 400, Loss: 0.6219
Iter: 500, Loss: 0.6151
Iter: 600, Loss: 0.6086
Iter: 700, Loss: 0.6025
Iter: 800, Loss: 0.5968
Iter: 900, Loss: 0.5913
Iter: 1000, Loss: 0.5862
Iter: 1100, Loss: 0.5813
Iter: 1200, Loss: 0.5767
Iter: 1300, Loss: 0.5723
Iter: 1400, Loss: 0.5682
Iter: 1500, Loss: 0.5643
Iter: 1600, Loss: 0.5606
Iter: 1700, Loss: 0.5571
Iter: 1800, Loss: 0.5537
Iter: 1900, Loss: 0.5506
Iter: 2000, Loss: 0.5476


In [45]:
test_x, test_y = x_train[1:2], y_train[1]

hypothesis  = logistic_regression(test_x).numpy()[0][0]
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = accuracy_fn(logistic_regression(test_x),test_y)
print("Hypothesis: ", hypothesis, "\nPredicted (Y) :", predicted.numpy(), "\nLable (Y) :", test_y)
print("Testset Accuracy: {:.4f}".format(accuracy))

Hypothesis:  0.76016605 
Predicted (Y) : 1.0 
Lable (Y) : [1.]
Testset Accuracy: 1.0000
